# Welcome to Enterprise-Scale!

## Verify Pre-req

Powershell Version > 7.0

In [ ]:
$psversiontable

Git Version > 2.24

In [ ]:
git --version

## Login to Azure

Clear Azure Context

In [ ]:
Clear-AzContext -Force

Login to Azure with SPN or User Account that has permission at '/' scope

In [ ]:
$user = ""
$password = ""
$tenantid = ""
$secureStringPwd = $password | ConvertTo-SecureString -AsPlainText -Force
$cred = New-Object System.Management.Automation.PSCredential -ArgumentList $user, $secureStringPwd
Connect-AzAccount -TenantId $tenantid  -ServicePrincipal -Credential $cred 


Verify SPN/user account  is logged in for the Tenant

In [ ]:
get-azcontext | fl

## Bootstrap new Tenant


Set GitHub token to access raw content

In [ ]:
$GitHubToken = 'AD4QREEEQ7XNHXIAN4IHMSK62YTRG'
Write-Output $GitHubToken

View Template File

In [ ]:
echo "https://raw.githubusercontent.com/Azure/CET-NorthStar/master/examples/Enterprise-Scale-Template-Deployment.json?token=$GitHubToken"

In [ ]:
(Invoke-WebRequest -Uri "https://raw.githubusercontent.com/Azure/CET-NorthStar/master/examples/Enterprise-Scale-Template-Deployment.json?token=$GitHubToken").Content | ConvertFrom-Json

Set Management Group Prefix

In [ ]:
$TopLevelManagementGroupPrefix = 'ES'
$TemplateParameterObject = @{'TopLevelManagementGroupPrefix'='ES'}

Initialize Tenant Deployment Parameter 

In [ ]:

$parameters = @{
    'Name'                    = 'Enterprise-Scale-Template'
    'Location'                = 'North Europe'
    'TemplateUri'             = "https://raw.githubusercontent.com/Azure/CET-NorthStar/master/examples/Enterprise-Scale-Template-Deployment.json?token=$GitHubToken"
    'TemplateParameterObject' = $TemplateParameterObject
    'Verbose'                 = $true
}


Invoke Tenant Level Deployment

In [ ]:
New-AzTenantDeployment @parameters

View Tenant Level Deployment

In [ ]:
Get-AzTenantDeployment | select DeploymentName, ProvisioningState, Timestamp,location |sort-object Timestamp -Descending

View Management Group Level Deployment

In [ ]:
Get-AzManagementGroupDeployment -ManagementGroupId $TopLevelManagementGroupPrefix | select DeploymentName, ProvisioningState, Timestamp |sort-object Timestamp -Descending

## Setting up Git

Ephermal space for Git

In [ ]:
jupyter --runtime-dir

Git Clone Your repo (Skip this step if you have already cloned).

Please Ensure your Git Credentails are available for PowerShell to use in your session.

In [ ]:
git clone https://github.com/uday31in/NorthStar.git

Change Path to Git Root

In [ ]:
Write-Host "Changing Current Directory to: $(jupyter --runtime-dir)\NorthStar"
cd "$(jupyter --runtime-dir)\NorthStar"

Add upstream repo

In [ ]:
git remote add upstream https://github.com/Azure/CET-NorthStar.git

Verify Remote

In [ ]:
git remote -v 

Pull latest upstream/master in your local master branch

In [ ]:
git pull upstream master -X theirs -f

## Initialize Enviornment

Ensure Current Path is set to Git Root of your repo

In [ ]:
Write-Host "Changing Current Directory to: $(jupyter --runtime-dir)\NorthStar"
cd "$(jupyter --runtime-dir)\NorthStar"

Ensure Azure Login

In [ ]:
Get-AzContext | fl

Import PowerShell Module

In [ ]:
Import-Module .\src\AzOps.psd1 -force
Get-ChildItem -Path .\src -Include *.ps1 -Recurse | ForEach-Object { .$_.FullName }

Intialize Git Repo for your Azure Enviornement.

Please Note: This will take few minutes to compelte depending size of an enviornment

In [ ]:
Initialize-AzOpsRepository -Verbose -SkipResourceGroup

Commit Change to Feaure Branch "initial-discovery"

In [ ]:
git checkout -b initial-discovery

Commit Changes to AzOps

In [ ]:
git add .\azops

View Git Status

In [ ]:
git status

Git commit

In [ ]:
git commit -m "Initializing Azure Enviornment"

Push your changes to your Git repo

In [ ]:
git push origin initial-discovery

Submit PR in Git Portal and merge to master before proceeding to next step

In [ ]:
git remote get-url --all origin

## Enable Git Action

Ensure Current Path is set to Git Root of your repo

In [ ]:
Write-Host "Changing Current Directory to: $(jupyter --runtime-dir)\NorthStar"
cd "$(jupyter --runtime-dir)\NorthStar"

Commit Change to Feaure Branch "initial-discovery"

In [ ]:
git checkout -b enable-git-action

Enable Action by copying ".github\workflows\azops-pull.yml.disabled" to ".github\workflows\azops-pull.yml"

In [ ]:
copy "$(jupyter --runtime-dir)\NorthStar\.github\workflows\azops-push.yml.disabled" "$(jupyter --runtime-dir)\NorthStar\.github\workflows\azops-push.yml"

Add File to Git

In [ ]:
git add .github\workflows\azops-push.yml

View Git Status

In [ ]:
git status

Git commit

In [ ]:
git commit -m "Enable Git Action"

Push your changes to your Git repo

In [ ]:
git push origin enable-git-action

Submit PR in Git Portal and merge to master

In [ ]:
git remote get-url --all origin

## Deploying New Policy Assignment using pipeline

Ensure Current Path is set to Git Root of your repo

In [ ]:
Write-Host "Changing Current Directory to: $(jupyter --runtime-dir)\NorthStar"
cd "$(jupyter --runtime-dir)\NorthStar"

Create Branch deploy-loganalytics

In [ ]:
git checkout -b deploy-loganalytics

View Policy Assignment

In [ ]:
echo 'https://github.com/Azure/CET-NorthStar/raw/master/azopsreference/3fc1081d-6105-4e19-b60c-1ec1252cf560/contoso/platform/management/.AzState/Microsoft.Authorization_policyAssignments-Deploy-Log-Analytics.parameters.json'

Create Policy Assignment Parameter file

In [ ]:
@"
{
  "`$schema": "http://schema.management.azure.com/schemas/2015-01-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "input": {
      "value": {
        "Name": "Deploy-Log-Analytics",
        "ResourceType": "Microsoft.Authorization/policyAssignments",
        "Location": "northeurope",        
        "Identity": {
          "type": "SystemAssigned"
        },
        "Properties": {
          "displayName": "Deploy-LogAnalytics",
          "policyDefinitionId": "/providers/Microsoft.Management/managementGroups/$($TopLevelManagementGroupPrefix)/providers/Microsoft.Authorization/policyDefinitions/Deploy-Log-Analytics",
          "scope": "/providers/Microsoft.Management/managementGroups/$($TopLevelManagementGroupPrefix)-management",
          "notScopes": [],
          "parameters": {
            "workspaceName": {
              "value": "$($TopLevelManagementGroupPrefix)-weu-la"
            },
            "automationAccountName": {
              "value": "$($TopLevelManagementGroupPrefix)-weu-aa"
            },
            "workspaceRegion": {
              "value": "westeurope"
            },
            "automationRegion": {
              "value": "westeurope"
            },
            "rgName": {
              "value": "$($TopLevelManagementGroupPrefix)-weu-mgmt"
            }
          },
          "enforcementMode": "Default"
        }
      }
    }
  }
}
"@  > ".\azops\Tenant Root Group\ES\ES-platform\ES-management\.AzState\Microsoft.Authorization_policyAssignments-Deploy-Log-Analytics.parameters.json"

Add File to Git

In [ ]:
git add ".\azops\Tenant Root Group\ES\ES-platform\ES-management\.AzState\Microsoft.Authorization_policyAssignments-Deploy-Log-Analytics.parameters.json"

View Git Status

In [ ]:
git status

Git commit

In [ ]:
git commit -m "Deploy Log Analytics Policy"

Push your changes to your Git repo

In [ ]:
git push origin deploy-loganalytics

Submit PR in Git Portal and wait for GitHub to action to complete.

DO NOT merge, Pull request to master branch before GitHub actions complete.

Go To Portal and verify Policy Assigment is created.

Pull Master barnach locally

In [ ]:
git checkout master && git pull

## Demo Drift Detection

<Manual> User Portal to make changes e.g. Add new management Group or update exisitng policy definition or assignment. 
    
To simulate OOB changes, we are making imperative change via PowerShell.

In [ ]:
$GroupName = "$TopLevelManagementGroupPrefix-IAB"
$ParentId  = "/providers/Microsoft.Management/managementGroups/$TopLevelManagementGroupPrefix"

New-AzManagementGroup -GroupName $GroupName -DisplayName $GroupName -ParentId $ParentId

Create Branch deploy-vWan

In [ ]:
git checkout -b deploy-vWan

View Policy Assignment

In [ ]:
echo 'https://github.com/Azure/CET-NorthStar/blob/master/azopsreference/3fc1081d-6105-4e19-b60c-1ec1252cf560/contoso/platform/connectivity/.AzState/Microsoft.Authorization_policyAssignments-Deploy-vWAN.parameters.json'

Create Policy Assignment Parameter file

In [ ]:
@"
{
  "`$schema": "http://schema.management.azure.com/schemas/2015-01-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "input": {
      "value": {
        "Name": "Deploy-VWAN",
        "ResourceType": "Microsoft.Authorization/policyAssignments",
        "Location": "northeurope",        
        "Identity": {
          "type": "SystemAssigned"
        },
        "Properties": {
          "displayName": "Deploy-vWAN",
          "policyDefinitionId": "/providers/Microsoft.Management/managementGroups/$($TopLevelManagementGroupPrefix)/providers/Microsoft.Authorization/policyDefinitions/Deploy-vWAN",
          "scope": "/providers/Microsoft.Management/managementGroups/$($TopLevelManagementGroupPrefix)-connectivity",
          "notScopes": [],
          "parameters": {
            "vwanname": {
              "value": "$($TopLevelManagementGroupPrefix)-vwan"
            },
            "vwanRegion": {
              "value": "northeurope"
            },
            "rgName": {
              "value": "$($TopLevelManagementGroupPrefix)-global-vwan"
            }
          },
          "description": "",
          "enforcementMode": "Default"
        }
      }
    }
  }
}
"@  > ".\azops\Tenant Root Group\ES\ES-platform\ES-connectivity\.AzState\Microsoft.Authorization_policyAssignments-Deploy-vWAN.parameters.json"

Add File to Git

In [ ]:
git add ".\azops\Tenant Root Group\ES\ES-platform\ES-connectivity\.AzState\Microsoft.Authorization_policyAssignments-Deploy-vWAN.parameters.json"

View Git Status

In [ ]:
git status

Git commit

In [ ]:
git commit -m "Deploy vWAN Policy"

Push your changes to your Git repo

In [ ]:
git push origin deploy-vWan

Submit PR in Git Portal and wait for GitHub to action to complete.

DO NOT merge, Pull request to master branch before GitHub actions complete.

When Git Action runs, it should detect 

## Clean-up Previous Install

Import-Module

In [ ]:
Import-Module .\src\AzOps.psd1 -force
Get-ChildItem -Path .\src -Include *.ps1 -Recurse | ForEach-Object { .$_.FullName }

Management Group To Clean-up

In [ ]:
$ManagementGroupPrefix = "ES"

Clean-up Management Group

In [ ]:
if (Get-AzManagementGroup -GroupName $ManagementGroupPrefix -ErrorAction SilentlyContinue) {
    Write-Verbose "Cleaning up Tailspin Management Group"
    Remove-AzOpsManagementGroup -groupName  $ManagementGroupPrefix -Verbose
}

Clean-up Tenant Deployment

If you see an error "Your Azure credentials have not been set up or have expired", please re-run command. It might take several retries.

In [ ]:
#Clean up Tenant Level Deployments
Get-AzTenantDeployment | Foreach-Object -Parallel { Remove-AzTenantDeployment -Name $_.DeploymentName -Confirm:$false}


Delete initial-discovery remote branch

In [ ]:
git branch -D initial-discovery

In [ ]:
git push origin --delete initial-discovery

Delete enable-git-action remote branch

In [ ]:
git branch -D enable-git-action

In [ ]:
git push origin --delete enable-git-action

Delete deploy-loganalytics remote branch

In [ ]:
git branch -D deploy-loganalytics

In [ ]:
git push origin --delete deploy-loganalytics

Delete deploy-loganalytics remote branch

In [ ]:
git branch -D deploy-vWAN

In [ ]:
git push origin --delete deploy-vWAN

Reset upstream master branch

In [ ]:
git checkout master -f 

In [ ]:
git pull upstream master
git reset --hard upstream/master

In [ ]:
git push -f

Remove Local Git Folder

In [ ]:
rm -recurse -force "$(jupyter --runtime-dir)\NorthStar"